В результате подготовительных операций был получен обучающий датасет изображений, который сохранён в папке **train**. Структура датанных разделена на подпапки **men** и **women**, каждая из которых в свою очередь имеет вложенные подпапки **eng** и **rus** для разделения знаменитостей на иностранных и отечественных. Для удобства подобной структуры планируется придерживаться во всём проекте.

Так как в идеале в готовом решении мы хотели бы получить сервис, который по одному загруженному изображению находит максимально похожих иностранных и отечественных знаменитостей, то первая проблема, которую предстоить решить - это автоматическое определение пола по загруженному изображению.

Для решения этой задачи необходимо обучить модель на определение пола по загруженному изображению. Для этого необходимо получить и сохранить для каждого изображения эмбединги на основе всё той же библиотеки *face_recognition* и сохранить их с сохранением структуры данных в папке **static**. 

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

from tqdm import notebook
import face_recognition
from PIL import Image
import numpy as np
import pickle
import glob
import os

### 1. Получение эмбедингов изображений

In [2]:
# Функция получения и сохранения эмбедингов

def get_embedings(gender='men'):  
    pet_path = './Documents/DataScience/PET-project'
    
    # Создание папки для хранения эмбедингов
    if os.path.exists(f'{pet_path}/static') != True:
        os.mkdir(f'{pet_path}/static')
        
    # Создание подпапок для актеров и актрис
    if os.path.exists(f'{pet_path}/static/{gender}') != True:
        os.mkdir(f'{pet_path}/static/{gender}')
        
    nation = ['eng', 'rus']
    for natio in nation:
        # Создание подпапок для разделения знаменитостей на иностранных и отечественных
        if os.path.exists(f'{pet_path}/static/{gender}/{natio}') != True:
            os.mkdir(f'{pet_path}/static/{gender}/{natio}')
        
        # Получаем список знаменитостей из папки train
        folders = glob.glob(f'{pet_path}/train/{gender}/{natio}/*')
        
        dict_labels = dict()
        for i, name in enumerate(folders):
            if natio == 'eng':
                name = name[name.find('eng\\') + 4:].strip()
            elif natio == 'rus':
                name = name[name.find('rus\\') + 4:].strip()
            dict_labels[name] = i
        
        # Сохранение словаря с метками
        pkl_patch = f'{pet_path}/static/'
        
        if gender == 'men':
            role = 'actors'
        elif gender == 'women':
            role = 'actresses'
        
        with open(pkl_patch + f'{natio}-{role}-dict-labels.pkl','wb') as f:
            pickle.dump(dict_labels, f)
        
        # Создаём пустые массивы под эмбеддинги и метки
        embedings = np.empty(128)
        target = []
        
        for person in notebook.tqdm(list(dict_labels.keys())):
            images = os.listdir(f'{pet_path}/train/{gender}/{natio}/{person}')
            
            for num, person_img in enumerate(images):
                face = face_recognition.load_image_file(f'{pet_path}/train/{gender}/{natio}/{person}/{person_img}')
                try:
                    # Преобразуем фото с лицом в вектор, получаем embeding
                    face_enc = face_recognition.face_encodings(face)[0]
                    
                    # Добавляем в датасет матрицу
                    embedings = np.vstack((embedings, face_enc))
                    
                    # Добавляем таргет по текущему индексу
                    target.append(dict_labels[person])
                except Exception as ex:
                    print(f'Error message {ex}')
                    
            print(f'Embeding for :: {person} - successfully received !')
        
        # Удаляем из датасета первый элемент, так как это пустая матрица
        embedings_ = embedings[1:]
        
        with open(f'{pet_path}/static/{gender}/{natio}/' + 'embedings-train.pkl','wb') as f:
            pickle.dump(embedings_, f)
        
        with open(f'{pet_path}/static/{gender}/{natio}/' + 'labels-train.pkl','wb') as f:
            pickle.dump(target, f)

In [20]:
get_embedings()

  0%|          | 0/2378 [00:00<?, ?it/s]

Embeding for :: 50 Cent - successfully received !
Embeding for :: A Martinez - successfully received !
Error message list index out of range
Embeding for :: Aamir Khan - successfully received !
Embeding for :: Aaron Eckhart - successfully received !
Embeding for :: Aaron Paul - successfully received !
Embeding for :: Aaron Pierre - successfully received !
Embeding for :: Aaron Taylor-Johnson - successfully received !
Embeding for :: Aasif Mandvi - successfully received !
Embeding for :: Abhishek Bachchan - successfully received !
Embeding for :: Adam Arkin - successfully received !
Embeding for :: Adam Baldwin - successfully received !
Embeding for :: Adam Beach - successfully received !
Embeding for :: Adam Brody - successfully received !
Embeding for :: Adam Devine - successfully received !
Error message list index out of range
Embeding for :: Adam DiMarco - successfully received !
Error message list index out of range
Error message list index out of range
Embeding for :: Adam Driver

Embeding for :: Anthony Zerbe - successfully received !
Error message list index out of range
Embeding for :: Anton Yelchin - successfully received !
Error message list index out of range
Error message list index out of range
Embeding for :: Antonio Banderas - successfully received !
Embeding for :: Antonio Sabato Jr - successfully received !
Embeding for :: Antony Starr - successfully received !
Embeding for :: Anupam Tripathi - successfully received !
Embeding for :: Anıl İlter - successfully received !
Embeding for :: Aras Aydin - successfully received !
Embeding for :: Aras Bulut Iynemli - successfully received !
Embeding for :: Archie Renaux - successfully received !
Embeding for :: Ari Millen - successfully received !
Embeding for :: Arifin Putra - successfully received !
Embeding for :: Arjun Gupta - successfully received !
Embeding for :: Armand Assante - successfully received !
Embeding for :: Armie Hammer - successfully received !
Embeding for :: Arnold Schwarzenegger - succe

Embeding for :: Brian Dennehy - successfully received !
Embeding for :: Brian J. Smith - successfully received !
Embeding for :: Brian Krause - successfully received !
Embeding for :: Brian Thompson - successfully received !
Embeding for :: Brian Tyree Henry - successfully received !
Embeding for :: Brion James - successfully received !
Embeding for :: Bruce Campbell - successfully received !
Embeding for :: Bruce Dern - successfully received !
Embeding for :: Bruce Greenwood - successfully received !
Embeding for :: Bruce Lee - successfully received !
Embeding for :: Bruce Spence - successfully received !
Embeding for :: Bruce Willis - successfully received !
Embeding for :: Bruno Cremer - successfully received !
Embeding for :: Bryan Cranston - successfully received !
Embeding for :: Bryan Greenberg - successfully received !
Embeding for :: Bud Spencer - successfully received !
Embeding for :: Burak Deniz - successfully received !
Embeding for :: Burak Tozkoparan - successfully recei

Embeding for :: Christopher Meloni - successfully received !
Embeding for :: Christopher Mintz-Plasse - successfully received !
Embeding for :: Christopher Plummer - successfully received !
Embeding for :: Christopher Reeve - successfully received !
Embeding for :: Christopher Walken - successfully received !
Embeding for :: Chuck Norris - successfully received !
Embeding for :: Ciarán Hinds - successfully received !
Embeding for :: Cillian Murphy - successfully received !
Embeding for :: Claes Bang - successfully received !
Embeding for :: Clancy Brown - successfully received !
Embeding for :: Clark Gable - successfully received !
Embeding for :: Clark Gregg - successfully received !
Embeding for :: Clarke Peters - successfully received !
Embeding for :: Claude Rains - successfully received !
Embeding for :: Clayton Watson - successfully received !
Embeding for :: Cliff Curtis - successfully received !
Embeding for :: Cliff Robertson - successfully received !
Embeding for :: Clifton C

...

Embeding for :: Виктор Вержбицкий - successfully received !
Error message list index out of range
Embeding for :: Виктор Добронравов - successfully received !
Embeding for :: Виктор Ильичев - successfully received !
Embeding for :: Виктор Логинов - successfully received !
Embeding for :: Виктор Павлов - successfully received !
Embeding for :: Виктор Петров - successfully received !
Embeding for :: Виктор Проскурин - successfully received !
Embeding for :: Виктор Раков - successfully received !
Embeding for :: Виктор Сергачев - successfully received !
Embeding for :: Виктор Смирнов - successfully received !
Embeding for :: Виктор Сухоруков - successfully received !
Embeding for :: Виктор Хориняк - successfully received !
Embeding for :: Вилен Бабичев - successfully received !
Embeding for :: Вилле Хаапасало - successfully received !
Embeding for :: Виталий Абдулов - successfully received !
Embeding for :: Виталий Гогунский - successfully received !
Embeding for :: Виталий Кищенко - succ

Embeding for :: Данила Поперечный - successfully received !
Embeding for :: Данила Якушев - successfully received !
Embeding for :: Даня Милохин - successfully received !
Embeding for :: Дельфин - successfully received !
Embeding for :: Демис Карибидис - successfully received !
Embeding for :: Денис Беспалый - successfully received !
Embeding for :: Денис Бузин - successfully received !
Embeding for :: Денис Васильев - successfully received !
Embeding for :: Денис Власенко - successfully received !
Embeding for :: Денис Косяков - successfully received !
Embeding for :: Денис Кукояка - successfully received !
Embeding for :: Денис Матросов - successfully received !
Embeding for :: Денис Рожков - successfully received !
Embeding for :: Денис Синявский - successfully received !
Embeding for :: Денис Сладков - successfully received !
Error message list index out of range
Embeding for :: Денис Шведов - successfully received !
Embeding for :: Джемал Тетруашвили - successfully received !
Erro

Embeding for :: Игорь Николаев - successfully received !
Embeding for :: Игорь Огурцов - successfully received !
Embeding for :: Игорь Петренко - successfully received !
Embeding for :: Игорь Ромащенко - successfully received !
Embeding for :: Игорь Савочкин - successfully received !
Embeding for :: Игорь Сигаев - successfully received !
Embeding for :: Игорь Скляр - successfully received !
Embeding for :: Игорь Стам - successfully received !
Embeding for :: Игорь Старосельцев - successfully received !
Embeding for :: Игорь Старыгин - successfully received !
Embeding for :: Игорь Угольников - successfully received !
Embeding for :: Игорь Филиппов - successfully received !
Embeding for :: Игорь Черневич - successfully received !
Embeding for :: Игорь Чехов - successfully received !
Embeding for :: Илья Бледный - successfully received !
Embeding for :: Илья Глинников - successfully received !
Embeding for :: Илья Ермолов - successfully received !
Embeding for :: Илья Ильиных - successful

Embeding for :: Михаил Хрусталёв - successfully received !
Embeding for :: Михаил Хуранов - successfully received !
Embeding for :: Михаил Черняк - successfully received !
Embeding for :: Мухтарбек Кантемиров - successfully received !
Embeding for :: Никита Волков - successfully received !
Embeding for :: Никита Джигурда - successfully received !
Embeding for :: Никита Дювбанов - successfully received !
Embeding for :: Никита Еленев - successfully received !
Error message list index out of range
Embeding for :: Никита Ефремов - successfully received !
Embeding for :: Никита Карпинский - successfully received !
Embeding for :: Никита Кологривый - successfully received !
Embeding for :: Никита Кукушкин - successfully received !
Embeding for :: Никита Михалков - successfully received !
Embeding for :: Никита Павленко - successfully received !
Embeding for :: Никита Панфилов - successfully received !
Embeding for :: Никита Подгорный - successfully received !
Embeding for :: Никита Пресняко

Embeding for :: Сергей Годин - successfully received !
Embeding for :: Сергей Горобченко - successfully received !
Embeding for :: Сергей Горошко - successfully received !
Embeding for :: Сергей Губанов - successfully received !
Embeding for :: Сергей Дружко - successfully received !
Embeding for :: Сергей Друзьяк - successfully received !
Embeding for :: Сергей Епишев - successfully received !
Embeding for :: Сергей Жарков - successfully received !
Embeding for :: Сергей Жигунов - successfully received !
Embeding for :: Сергей Иванов - successfully received !
Embeding for :: Сергей Колос - successfully received !
Embeding for :: Сергей Комаров - successfully received !
Embeding for :: Сергей Кристовский - successfully received !
Embeding for :: Сергей Куницкий - successfully received !
Embeding for :: Сергей Куприянов - successfully received !
Error message list index out of range
Embeding for :: Сергей Лавыгин - successfully received !
Embeding for :: Сергей Лукьянов - successfully r

In [21]:
get_embedings(gender='women')

  0%|          | 0/2329 [00:00<?, ?it/s]

Embeding for :: A.J. Cook - successfully received !
Embeding for :: Aaliyah - successfully received !
Embeding for :: Aarti Mann - successfully received !
Embeding for :: Abbey Lee - successfully received !
Error message list index out of range
Embeding for :: Abbie Cornish - successfully received !
Embeding for :: Abella Danger - successfully received !
Embeding for :: Abigail Breslin - successfully received !
Embeding for :: Abigail Cowen - successfully received !
Embeding for :: Abigail Spencer - successfully received !
Embeding for :: Ada Nicodemou - successfully received !
Embeding for :: Addison Rae - successfully received !
Embeding for :: Addison Timlin - successfully received !
Error message list index out of range
Embeding for :: Adelaide Kane - successfully received !
Embeding for :: Adele - successfully received !
Embeding for :: Adina Porter - successfully received !
Embeding for :: Adjoa Andoh - successfully received !
Embeding for :: Adria Arjona - successfully received 

Error message list index out of range
Error message list index out of range
Embeding for :: Amy Brenneman - successfully received !
Embeding for :: Amy Forsyth - successfully received !
Error message list index out of range
Embeding for :: Amy Poehler - successfully received !
Embeding for :: Amy Ryan - successfully received !
Embeding for :: Amy Seimetz - successfully received !
Error message list index out of range
Embeding for :: Amy Smart - successfully received !
Embeding for :: Amy Tyger - successfully received !
Embeding for :: Amy Yasbeck - successfully received !
Embeding for :: Amybeth McNulty - successfully received !
Embeding for :: An Hee-yeon - successfully received !
Embeding for :: Ana de Armas - successfully received !
Error message list index out of range
Embeding for :: Ana de la Reguera - successfully received !
Embeding for :: Ana Ularu - successfully received !
Embeding for :: Anamaria Vartolomei - successfully received !
Embeding for :: Anastasia Tsilimpiou - suc

Embeding for :: Barbie Ferreira - successfully received !
Embeding for :: Barbra Streisand - successfully received !
Embeding for :: Basak Gümülcinelioglu - successfully received !
Embeding for :: Beanie Feldstein - successfully received !
Embeding for :: Beata Tyszkiewicz - successfully received !
Embeding for :: Beatie Edney - successfully received !
Embeding for :: Beau Garrett - successfully received !
Embeding for :: Becki Newton - successfully received !
Embeding for :: Bel Powley - successfully received !
Embeding for :: Belinda Bromilow - successfully received !
Embeding for :: Belinda McClory - successfully received !
Embeding for :: Bella Heathcote - successfully received !
Error message list index out of range
Error message list index out of range
Embeding for :: Bella Thorne - successfully received !
Embeding for :: Belén Rueda - successfully received !
Embeding for :: Bensu Soral - successfully received !
Embeding for :: Beren Saat - successfully received !
Embeding for ::

Embeding for :: Carrie Coon - successfully received !
Embeding for :: Carrie Fisher - successfully received !
Error message list index out of range
Embeding for :: Carrie Preston - successfully received !
Error message list index out of range
Embeding for :: Carrie-Anne Moss - successfully received !
Embeding for :: Carroll Baker - successfully received !
Embeding for :: Carré Otis - successfully received !
Embeding for :: Casca Akashova - successfully received !
Embeding for :: Cassandra Peterson - successfully received !
Embeding for :: Cassandra Scerbo - successfully received !
Embeding for :: Cassie Clare - successfully received !
Error message list index out of range
Error message list index out of range
Embeding for :: Cate Blanchett - successfully received !
Embeding for :: Caterina Murino - successfully received !
Embeding for :: Caterina Scorsone - successfully received !
Error message list index out of range
Embeding for :: Catherine Bell - successfully received !
Error messa

...

Embeding for :: Кристина Орбакайте - successfully received !
Embeding for :: Кристина Поли - successfully received !
Embeding for :: Ксения Алферова - successfully received !
Embeding for :: Ксения Асмоловская - successfully received !
Embeding for :: Ксения Бржезовская - successfully received !
Embeding for :: Ксения Буравская - successfully received !
Error message list index out of range
Embeding for :: Ксения Зуева - successfully received !
Embeding for :: Ксения Каталымова - successfully received !
Embeding for :: Ксения Кузнецова - successfully received !
Embeding for :: Ксения Кутепова - successfully received !
Embeding for :: Ксения Лаврова-Глинка - successfully received !
Embeding for :: Ксения Лукьянчикова - successfully received !
Embeding for :: Ксения Мишина - successfully received !
Embeding for :: Ксения Непотребная - successfully received !
Embeding for :: Ксения Радченко - successfully received !
Embeding for :: Ксения Раппопорт - successfully received !
Embeding for :

Embeding for :: Мария Ивакова - successfully received !
Embeding for :: Мария Иващенко - successfully received !
Embeding for :: Мария Ильюхина - successfully received !
Embeding for :: Мария Киселева - successfully received !
Embeding for :: Мария Климова - successfully received !
Embeding for :: Мария Кожевникова - successfully received !
Embeding for :: Мария Кравченко - successfully received !
Embeding for :: Мария Куликова - successfully received !
Embeding for :: Мария Кунах - successfully received !
Embeding for :: Мария Лисовая - successfully received !
Embeding for :: Мария Луговая - successfully received !
Embeding for :: Мария Мацель - successfully received !
Embeding for :: Мария Машкова - successfully received !
Embeding for :: Мария Милютина - successfully received !
Embeding for :: Мария Миногарова - successfully received !
Embeding for :: Мария Миронова - successfully received !
Embeding for :: Мария Овчинникова - successfully received !
Embeding for :: Мария Пирогова -

Embeding for :: Нюша Шурочкина - successfully received !
Embeding for :: Оксана Акиньшина - successfully received !
Error message list index out of range
Embeding for :: Оксана Базилевич - successfully received !
Embeding for :: Оксана Кутузова - successfully received !
Embeding for :: Оксана Лаврентьева - successfully received !
Embeding for :: Оксана Сурнина - successfully received !
Error message list index out of range
Embeding for :: Оксана Фёдорова - successfully received !
Error message list index out of range
Embeding for :: Оксана Фандера - successfully received !
Embeding for :: Олеся Власова - successfully received !
Embeding for :: Олеся Грибок - successfully received !
Embeding for :: Олеся Железняк - successfully received !
Embeding for :: Олеся Жураковская - successfully received !
Embeding for :: Олеся Судзиловская - successfully received !
Error message list index out of range
Embeding for :: Олеся Фаттахова - successfully received !
Embeding for :: Ольга Арнтгольц - s

Embeding for :: Светлана Смирнова - successfully received !
Embeding for :: Светлана Смирнова-Кацагаджиева - successfully received !
Embeding for :: Светлана Смирнова-Марцинкевич - successfully received !
Embeding for :: Светлана Степанковская - successfully received !
Embeding for :: Светлана Суханова - successfully received !
Embeding for :: Светлана Тома - successfully received !
Embeding for :: Светлана Устинова - successfully received !
Error message list index out of range
Embeding for :: Светлана Ходченкова - successfully received !
Embeding for :: Светлана Чуйкина - successfully received !
Embeding for :: Светлана Щедрина - successfully received !
Embeding for :: Северия Янушаускайте - successfully received !
Embeding for :: Серафима Низовская - successfully received !
Embeding for :: Серафима Огарёва - successfully received !
Embeding for :: Сесиль Плеже - successfully received !
Embeding for :: Сесиль Свердлова - successfully received !
Embeding for :: Снежана Самохина - succ

Embeding for :: Яна Крайнова - successfully received !
Embeding for :: Яна Науменко - successfully received !
Embeding for :: Яна Поплавская - successfully received !
Error message list index out of range
Embeding for :: Яна Сексте - successfully received !
Embeding for :: Яна Смоленцева - successfully received !
Embeding for :: Яна Троянова - successfully received !
Embeding for :: Яна Шивкова - successfully received !
Embeding for :: Янина Бугрова - successfully received !
Embeding for :: Янина Жеймо - successfully received !
Embeding for :: Янина Лисовская - successfully received !
Embeding for :: Янина Соколовская - successfully received !
Error message list index out of range
Embeding for :: Янина Студилина - successfully received !
Embeding for :: Ясмина Омерович - successfully received !


### 2. Обучение модели для определения пола по загруженному изображению

После того, как получены эмбединги обучим модель для определения пола по загруженному изображению. В качестве бейзлайна попробуем использовать линенейную регрессию.

In [3]:
# Функция для считывания сохраненных эмбедингов

def read_embedings(gender='men'):
    stat_path = './Documents/DataScience/PET-project/static'
    
    nation = ['eng', 'rus']
    target = []
    for natio in nation:
        with open(f'{stat_path}/{gender}/{natio}/embedings-train.pkl','rb') as f:
            if natio == 'eng':
                embedings = pickle.load(f)
            else:
                embedings = np.concatenate((embedings, pickle.load(f)), axis=0)
        
        with open(f'{stat_path}/{gender}/{natio}/labels-train.pkl','rb') as f:
            target += pickle.load(f)
    
    return (embedings, target)

In [140]:
# Сохраняем эмбединги и метки для женщин в переменных

read_embedings_women = read_embedings(gender='women')
embedings_women = read_embedings_women[0]
target_women = read_embedings_women[1]

In [141]:
# Проверяем соотвествие количества эмбедингов и меток для женщин

len(embedings_women), len(target_women)

(51649, 51649)

In [143]:
# Сохраняем эмбединги и метки для мужчин в переменных

read_embedings_men = read_embedings()
embedings_men = read_embedings_men[0]
target_men = read_embedings_men[1]

In [144]:
# Проверяем соотвествие количества эмбедингов и меток для мужчин

len(embedings_men), len(target_men)

(34389, 34389)

In [145]:
# Объединяем метки мужчин и женщин пометив всех женщин 1, а всех мужчин 0. также объединяем их эмбединги. Ещё раз проверяем
# количество.

target = [1 for i in target_women] + [0 for i in target_men]
embedings = np.concatenate((embedings_women, embedings_men), axis=0)
len(embedings), len(target)

(86038, 86038)

In [162]:
# Сплитим датасет на тренировочную и проверочную части

X_train, X_test, y_train, y_test = train_test_split(embedings, target, test_size=0.2, random_state=21)

In [163]:
%%time
model = LogisticRegression()
model.fit(X_train, y_train)

Wall time: 1.08 s


LogisticRegression()

In [164]:
f1 = f1_score(y_test, model.predict(X_test))
print(f'F1 score = {f1}')

F1 score = 0.9944852941176472


F-мера для обученной модели на определение пола приближается к 100%. Поэтому, считаем задача определения пола решена. Сохраняем обученную модель для последующего использования в продакшине.

In [165]:
pkl_patch = './Documents/DataScience/PET-project/static/models'

if os.path.exists(pkl_patch) != True:
    os.mkdir(pkl_patch)

with open(f'{pkl_patch}/model_gender_determination.pkl','wb') as f:
    pickle.dump(model, f)

Далее обучим модели на поиск лиц по имеющемуся в папке **target** датасету.

### 3. Обучение моделей на распознавание лиц

Согласно поставленной задаче необходимо обучить четыре модели:
* иностранных актёров;
* отечественных актёров;
* иностранных актрис;
* отечественных актрис.

Обученные модели сохранить для последующего использования в продакшине.

#### Модель для иностранных актёров

In [32]:
# Подгрузим сохранённые эмбединги иностранных актёров для считывания

pkl_patch = './Documents/DataScience/PET-project/static/men/eng'

with open(f'{pkl_patch}/embedings-train.pkl','rb') as f:
    embedings_ = pickle.load(f)
    
with open(f'{pkl_patch}/labels-train.pkl','rb') as f:
    target = pickle.load(f)

In [33]:
# Проверяем соотвествие количества эмбедингов и меток

embedings_.shape, len(target)

((27415, 128), 27415)

In [34]:
# Функция поиска индексов знаменитостей с одним изображеним
less_one = []

for index in range(len(target)):
    cnt = target.count(target[index])
    if cnt < 2:
        less_one.append(index)

In [35]:
# Удаление знаменитостей с одним изображением из эмбедингов и таргетов

embedings = []
for index in range(len(embedings_)):
    if index not in less_one:
        embedings.append(embedings_[index])
        
target_ = []
for index in range(len(target)):
    if index not in less_one:
        target_.append(target[index])

In [36]:
len(target_)

27408

In [37]:
# Сплитим датасет на тренировочную и проверочную части

X_train, X_test, y_train, y_test = train_test_split(embedings, target_, test_size=0.25, stratify=target_, random_state=21)

Линейную регрессию будем использовать с решающим алгоритмом *solver='liblinear'* и коэффициентом регуляризации *C=50*.

In [38]:
%%time
model = LogisticRegression(solver='liblinear', C=50)
model.fit(X_train, y_train)

Wall time: 37min 32s


LogisticRegression(C=50, solver='liblinear')

In [39]:
f1 = f1_score(y_test, model.predict(X_test), average='micro')
print(f'F1 score = {f1}')

F1 score = 0.9619089316987741


In [40]:
# Сохранение модели
"""
pkl_patch = './Documents/DataScience/PET-project/static/models'

with open(f'{pkl_patch}/eng_actors.pkl','wb') as f:
    pickle.dump(model, f)"""

#### Модель для отечественных актёров

In [193]:
# Подгрузим сохранённые эмбединги отечественных актёров для считывания

pkl_patch = './Documents/DataScience/PET-project/static/men/rus'

with open(f'{pkl_patch}/embedings-train.pkl','rb') as f:
    embedings_ = pickle.load(f)
    
with open(f'{pkl_patch}/labels-train.pkl','rb') as f:
    target = pickle.load(f)

In [194]:
# Проверяем соотвествие количества эмбедингов и меток

embedings_.shape, len(target)

((6974, 128), 6974)

In [195]:
# Функция поиска индексов знаменитостей с одним изображеним
less_one = []

for index in range(len(target)):
    cnt = target.count(target[index])
    if cnt < 2:
        less_one.append(index)

In [197]:
# Удаление знаменитостей с одним изображением из эмбедингов и таргетов

embedings = []
for index in range(len(embedings_)):
    if index not in less_one:
        embedings.append(embedings_[index])
        
target_ = []
for index in range(len(target)):
    if index not in less_one:
        target_.append(target[index])

In [201]:
# Сплитим датасет на тренировочную и проверочную части

X_train, X_test, y_train, y_test = train_test_split(embedings, target_, test_size=0.275, stratify=target_, random_state=21)

In [203]:
%%time
model = LogisticRegression(solver='liblinear', C=50)
model.fit(X_train, y_train)

Wall time: 2min 42s


LogisticRegression(C=50, solver='liblinear')

In [204]:
f1 = f1_score(y_test, model.predict(X_test), average='micro')
print(f'F1 score = {f1}')

F1 score = 0.9655891553701773


In [205]:
# Сохранение модели
"""
pkl_patch = './Documents/DataScience/PET-project/static/models'

with open(f'{pkl_patch}/rus_actors.pkl','wb') as f:
    pickle.dump(model, f)"""

#### Модель для иностранных актрис

In [41]:
# Подгрузим сохранённые эмбединги иностранных актрис для считывания

pkl_patch = './Documents/DataScience/PET-project/static/women/eng'

with open(f'{pkl_patch}/embedings-train.pkl','rb') as f:
    embedings_ = pickle.load(f)
    
with open(f'{pkl_patch}/labels-train.pkl','rb') as f:
    target = pickle.load(f)

In [42]:
# Проверяем соотвествие количества эмбедингов и меток

embedings_.shape, len(target)

((38509, 128), 38509)

In [43]:
# Функция поиска индексов знаменитостей с одним изображеним
less_one = []

for index in range(len(target)):
    cnt = target.count(target[index])
    if cnt < 2:
        less_one.append(index)

In [44]:
# Удаление знаменитостей с одним изображением из эмбедингов и таргетов

embedings = []
for index in range(len(embedings_)):
    if index not in less_one:
        embedings.append(embedings_[index])
        
target_ = []
for index in range(len(target)):
    if index not in less_one:
        target_.append(target[index])

In [45]:
# Сплитим датасет на тренировочную и проверочную части

X_train, X_test, y_train, y_test = train_test_split(embedings, target_, test_size=0.25, stratify=target_, random_state=21)

In [46]:
%%time
model = LogisticRegression(solver='liblinear', C=50)
model.fit(X_train, y_train)

Wall time: 46min 11s


LogisticRegression(C=50, solver='liblinear')

In [47]:
f1 = f1_score(y_test, model.predict(X_test), average='micro')
print(f'F1 score = {f1}')

F1 score = 0.9641632907447804


In [48]:
# Сохранение модели
"""
pkl_patch = './Documents/DataScience/PET-project/static/models'

with open(f'{pkl_patch}/eng_actresses.pkl','wb') as f:
    pickle.dump(model, f)"""

#### Модель для отечественных актрис

In [23]:
# Подгрузим сохранённые эмбединги отечественных актрис для считывания

pkl_patch = './Documents/DataScience/PET-project/static/women/rus'

with open(f'{pkl_patch}/embedings-train.pkl','rb') as f:
    embedings_ = pickle.load(f)
    
with open(f'{pkl_patch}/labels-train.pkl','rb') as f:
    target = pickle.load(f)

In [24]:
# Проверяем соотвествие количества эмбедингов и меток

embedings_.shape, len(target)

((13140, 128), 13140)

In [25]:
# Функция поиска индексов знаменитостей с одним изображеним
less_one = []

for index in range(len(target)):
    cnt = target.count(target[index])
    if cnt < 2:
        less_one.append(index)

In [26]:
less_one

[4894]

In [27]:
# Удаление знаменитостей с одним изображением из эмбедингов и таргетов

embedings = []
for index in range(len(embedings_)):
    if index not in less_one:
        embedings.append(embedings_[index])
        
target_ = []
for index in range(len(target)):
    if index not in less_one:
        target_.append(target[index])

In [28]:
# Сплитим датасет на тренировочную и проверочную части

X_train, X_test, y_train, y_test = train_test_split(embedings, target_, test_size=0.275, stratify=target_, random_state=21)

In [29]:
%%time
model = LogisticRegression(solver='liblinear', C=50)
model.fit(X_train, y_train)

Wall time: 5min 56s


LogisticRegression(C=50, solver='liblinear')

In [30]:
f1 = f1_score(y_test, model.predict(X_test), average='micro')
print(f'F1 score = {f1}')

F1 score = 0.9717764250138351


In [31]:
# Сохранение модели
"""
pkl_patch = './Documents/DataScience/PET-project/static/models'

with open(f'{pkl_patch}/rus_actresses.pkl','wb') as f:
    pickle.dump(model, f)"""

После обучения F-мера для модели иностранных актёров составила 96,19%, для отечественных актёров - 96,56%, для иностранных актрис - 96,42%, для отечественных актрис - 97,17%. Будем использовать эти модели в продакшине.